In [ ]:
import pandas as pd
import pyarrow as pya
import fastparquet as fp
import datetime as dt

In [52]:
import_file = '../data/yellow_tripdata_2023-01.parquet'

df = pd.read_parquet(import_file, engine= 'fastparquet')

uber_df = df.loc[(df['tpep_pickup_datetime'] >= '2023-01-01') & (df['tpep_pickup_datetime'] <= '2023-01-05')]


In [53]:
uber_df.to_csv ('../data/yellow_tripdata_2023-01_05.csv')

Check if all the fields of DF are in correct format or need to be converted

In [ ]:
uber_df.info()

Drop duplicates from the data to create fact and dims

In [ ]:
uber_df = uber_df.drop_duplicates().reset_index(drop=True)
uber_df ['trip_id'] = uber_df.index

In [ ]:
uber_df.head()

Create Dimension tables

In [ ]:
dim_datetime = uber_df[['tpep_pickup_datetime','tpep_dropoff_datetime']].drop_duplicates().reset_index(drop=True)

# Add other needed columns:

dim_datetime['pick_hour'] = dim_datetime['tpep_pickup_datetime'].dt.hour
dim_datetime['pick_day'] = dim_datetime['tpep_pickup_datetime'].dt.day
dim_datetime['pick_month'] = dim_datetime['tpep_pickup_datetime'].dt.month
dim_datetime['pick_year'] = dim_datetime['tpep_pickup_datetime'].dt.year
dim_datetime['pick_weekday'] = dim_datetime['tpep_pickup_datetime'].dt.weekday
dim_datetime['drop_hour'] = dim_datetime['tpep_pickup_datetime'].dt.hour
dim_datetime['drop_day'] = dim_datetime['tpep_pickup_datetime'].dt.day
dim_datetime['drop_month'] = dim_datetime['tpep_pickup_datetime'].dt.month
dim_datetime['drop_year'] = dim_datetime['tpep_pickup_datetime'].dt.year
dim_datetime['drop_weekday'] = dim_datetime['tpep_pickup_datetime'].dt.weekday
dim_datetime['drop_weekday'] = dim_datetime['tpep_pickup_datetime'].dt.weekday

#PK
dim_datetime ['datetime_id'] = dim_datetime.index + 1

Reset column order of the dataframe

In [ ]:
dim_datetime = dim_datetime[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday', 'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]
dim_datetime.head()

In [ ]:
dim_passenger_count = uber_df[['passenger_count']].drop_duplicates().reset_index(drop=True)
dim_passenger_count['passenger_count_id'] = dim_passenger_count.index + 1
dim_passenger_count = dim_passenger_count[['passenger_count_id','passenger_count']]

dim_passenger_count.head()

In [ ]:
dim_trip_distance = uber_df[['trip_distance']].drop_duplicates().reset_index(drop=True)
dim_trip_distance['trip_distance_id'] = dim_trip_distance.index + 1
dim_trip_distance = dim_trip_distance[['trip_distance_id','trip_distance']]

dim_trip_distance.head()

In [ ]:
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

dim_ratecode = uber_df[['RatecodeID']].drop_duplicates().reset_index(drop=True)
dim_ratecode['rate_code_id'] = dim_ratecode.index + 1 #PK
dim_ratecode['rate_code_name'] = dim_ratecode ['RatecodeID'].map(rate_code_type)
dim_ratecode = dim_ratecode [['rate_code_id','RatecodeID','rate_code_name']]
dim_ratecode.head()

In [ ]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
dim_payment_type = uber_df[['payment_type']].drop_duplicates().reset_index(drop=True)
dim_payment_type ['payment_type_id'] = dim_payment_type.index + 1
dim_payment_type ['payment_type_name'] = dim_payment_type['payment_type'].map(payment_type_name)
dim_payment_type = dim_payment_type [['payment_type_id','payment_type','payment_type_name']]
dim_payment_type.head()

In [ ]:
dim_pick_location = uber_df [['PULocationID']].drop_duplicates().reset_index(drop=True)
dim_pick_location['pick_location_id'] = dim_pick_location.index + 1
dim_pick_location = dim_pick_location [['pick_location_id','PULocationID']]
dim_pick_location.head()

# pulocation_id		Location

In [ ]:
dim_drop_location = uber_df [['DOLocationID']].drop_duplicates().reset_index(drop=True)
dim_drop_location['drop_location_id'] = dim_drop_location.index + 1
dim_drop_location = dim_drop_location [['drop_location_id','DOLocationID']]
dim_drop_location.head()

In [ ]:
vendor_name = {
    1:'Creative Mobile Technologies',
    2:'VeriFone Inc.'
}

dim_vendor = uber_df [['VendorID']].drop_duplicates().reset_index(drop=True)
dim_vendor['vendor_id'] = dim_vendor.index + 1
dim_vendor ['vendor_name'] = dim_vendor['VendorID'].map(vendor_name)
dim_vendor = dim_vendor[['vendor_id','VendorID','vendor_name']]
dim_vendor.head()


In [ ]:
fact_table = uber_df.merge (dim_passenger_count,on='passenger_count')\
                    .merge (dim_trip_distance, on='trip_distance')\
                    .merge (dim_ratecode, on='RatecodeID')\
                    .merge (dim_payment_type, on='payment_type')\
                    .merge (dim_pick_location, on='PULocationID')\
                    .merge (dim_drop_location, on='DOLocationID')\
                    .merge (dim_datetime, on=['tpep_pickup_datetime','tpep_dropoff_datetime'])\
                    .merge (dim_vendor, on='VendorID')\
                    [['vendor_id', 'datetime_id', 'pick_location_id', 'drop_location_id', 'passenger_count_id', 'rate_code_id',
                        'payment_type_id', 'trip_distance_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
                        'improvement_surcharge', 'congestion_surcharge', 'airport_fee']]

# VendorID	tpep_pickup_datetime	tpep_dropoff_datetime	passenger_count	trip_distance	RatecodeID	store_and_fwd_flag	PULocationID	DOLocationID	payment_type	fare_amount	extra	mta_tax	tip_amount

In [ ]:
fact_table.head()